In [ ]:
import os
import shutil
import zipfile
import sys
import requests
import concurrent.futures
import urllib3
import sqlite3
import pandas as pd
from prettytable import PrettyTable

download_dir = './tmp/'
data_dir = './data/'
extract_dir = './data/raw/'
database_dir = './data/database.db'

# Download Data

In [ ]:
# Desabilita os avisos de certificado SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

urls = [
  'https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2019/remuneracao_media_docentes/remuneracao_docentes_brasil_2019.zip',
  'https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2019/remuneracao_media_docentes/remuneracao_docentes_uf_2019.zip',
  'https://download.inep.gov.br/informacoes_estatisticas/indicadores_educacionais/2019/remuneracao_media_docentes/remuneracao_docentes_municipios_2019.zip',
  'https://download.inep.gov.br/educacao_basica/portal_ideb/planilhas_para_download/2019/divulgacao_brasil_ideb_2019.zip',
  'https://download.inep.gov.br/educacao_basica/portal_ideb/planilhas_para_download/2019/divulgacao_regioes_ufs_ideb_2019.zip',
  'https://download.inep.gov.br/educacao_basica/portal_ideb/planilhas_para_download/2019/divulgacao_anos_iniciais_municipios_2019.zip',
  'https://download.inep.gov.br/educacao_basica/portal_ideb/planilhas_para_download/2019/divulgacao_anos_finais_municipios_2019.zip',
  'https://download.inep.gov.br/educacao_basica/portal_ideb/planilhas_para_download/2019/divulgacao_ensino_medio_municipios_2019.zip',
  'https://download.inep.gov.br/educacao_basica/portal_ideb/planilhas_para_download/2019/divulgacao_anos_iniciais_escolas_2019.zip',
  'https://download.inep.gov.br/educacao_basica/portal_ideb/planilhas_para_download/2019/divulgacao_anos_finais_escolas_2019.zip',
  'https://download.inep.gov.br/educacao_basica/portal_ideb/planilhas_para_download/2019/divulgacao_ensino_medio_escolas_2019.zip'
]

if os.path.exists(download_dir):
  shutil.rmtree(download_dir)
    
os.makedirs(download_dir, exist_ok=True)

if os.path.exists(data_dir):
  shutil.rmtree(data_dir)
    
os.makedirs(data_dir, exist_ok=True)

if os.path.exists(extract_dir):
  shutil.rmtree(extract_dir)
    
os.makedirs(extract_dir, exist_ok=True)

def download_file(url):
    filename = os.path.join(download_dir, os.path.basename(url))
    if 'google.colab' in sys.modules:
      # Se estiver no Google Colab, usa o comando !wget
      !wget -P tmp/ --no-check-certificate {url}
    else:
      with requests.get(url, stream=True, verify=False) as r:
        with open(filename, 'wb') as f:
          shutil.copyfileobj(r.raw, f)
      return filename

if __name__ == "__main__":
  with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_url = {executor.submit(download_file, url): url for url in urls}
    for future in concurrent.futures.as_completed(future_to_url):
      url = future_to_url[future]
      try:
        filename = future.result()
        print(f"Downloaded {url} to {filename}")
      except Exception as e:
        print(f"Failed to download {url}: {e}")

  for filename in os.listdir(download_dir):
    shutil.move(os.path.join(download_dir, filename), os.path.join(extract_dir, filename))

  for filename in os.listdir(extract_dir):
    if filename.endswith(".zip"):
      zip_path = os.path.join(extract_dir, filename)
      with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        found = False
        for file_name in zip_ref.namelist():
          if file_name.endswith(".xlsx"):
            dest_filename = os.path.basename(file_name)
            dest_path = os.path.join(extract_dir, dest_filename)
            with zip_ref.open(file_name) as zip_file, open(dest_path, 'wb') as extracted_file:
              extracted_file.write(zip_file.read())
            print(f'File {file_name} extracted to {extract_dir}')
            found = True

        if not found:
          print(f'Error: No .xlsx files found in the ZIP file {zip_path}')

      os.remove(zip_path)
      print(f'ZIP file {zip_path} removed')
    else:
      print(f'The file {filename} is not a valid ZIP file')


# Database creation

In [ ]:
region_column = 'region'
state_column = 'UF'
city_id_column = 'city_id'
city_name_column = 'city_name'
adm_dependence_column = 'adm_dependence'
education_column = 'teacher_education'
teacher_number_column = 'teacher_quantity'
rais_teacher_percentage_column = 'rais_teacher_percentage'
first_quartile_column = 'first_quartile'
median_column ='median'
average_column = 'average'
third_quartile_column = 'third_quartile',
standard_deviation_column = 'standard_deviation'
weekly_workload_column ='weekly_workload'
standardized_pay_40_hours_column ='standardized_pay_40_hours'

state_or_region = 'state_or_region'

school_id_column = 'school_id'
school_name_column = 'school_name'
school_type_column = 'school_type'

first_to_fifth_grade_results_column = '1st_to_5th_grade_results'
first_grade_results_column = '1st_grade_results'
second_grade_results_column = '2nd_grade_results'
third_grade_results_column = '3rd_grade_results'
fourth_grade_results_column = '4th_grade_results'
fifth_grade_results_column = '5th_grade_results'

sixth_to_nineth_grade_results_column = '6th_to_9th_grade_results'
sixth_grade_results_column = '6th_grade_results'
seventh_grade_results_column = '7th_grade_results'
eighth_grade_results_column = '8th_grade_results'
ninth_grade_results_column = '9th_grade_results'

first_to_third_high_grade_results_column = '1st_to_3rd_grade_results'
first_high_grade_results_column = '1st_grade_results'
second_high_grade_results_column = '2nd_grade_results'
third_high_grade_results_column = '3rd_grade_results'

yield_indicator_column = 'yield_indicator' # P
math_results_column = 'math_results'
portuguese_results_column = 'portuguese_results'
standardized_average_column = 'standardized_average' # N
ideb_column = 'IDEB_N_x_P'


# Verificando os arquivos extraídos
extracted_files = os.listdir(extract_dir)
extracted_files = sorted(extracted_files)

# Conectar ao banco de dados SQLite (isso criará o arquivo exemplo.db se não existir)
conn = sqlite3.connect(database_dir)


columns = [

    [
        adm_dependence_column,
        education_column, teacher_number_column, rais_teacher_percentage_column, first_quartile_column,
        median_column, average_column, third_quartile_column, standard_deviation_column, weekly_workload_column,
        standardized_pay_40_hours_column
    ], # Brasil columns

    [
        region_column, state_column, city_id_column, city_name_column, adm_dependence_column,
        education_column, teacher_number_column, rais_teacher_percentage_column, first_quartile_column,
        median_column, average_column, third_quartile_column, standard_deviation_column, weekly_workload_column,
        standardized_pay_40_hours_column
    ], # Municipios columns

    [
        region_column, state_column, adm_dependence_column,
        education_column, teacher_number_column, rais_teacher_percentage_column, first_quartile_column,
        median_column, average_column, third_quartile_column, standard_deviation_column, weekly_workload_column,
        standardized_pay_40_hours_column
    ], # UFs columns

    [
        state_column, city_id_column, city_name_column,
        school_id_column, school_name_column, school_type_column, sixth_to_nineth_grade_results_column, sixth_grade_results_column,
        seventh_grade_results_column, eighth_grade_results_column, ninth_grade_results_column, yield_indicator_column,
        math_results_column, portuguese_results_column, standardized_average_column, ideb_column
    ],
    [
        state_column, city_id_column, city_name_column,
        school_type_column, sixth_to_nineth_grade_results_column, sixth_grade_results_column,
        seventh_grade_results_column, eighth_grade_results_column, ninth_grade_results_column, yield_indicator_column,
        math_results_column, portuguese_results_column, standardized_average_column, ideb_column
    ],

    [
        state_column, city_id_column, city_name_column, school_id_column, school_name_column,
        school_type_column, first_to_fifth_grade_results_column, first_grade_results_column, second_grade_results_column,
        third_grade_results_column, fourth_grade_results_column, fifth_grade_results_column, yield_indicator_column,
        math_results_column, portuguese_results_column, standardized_average_column, ideb_column
    ],
    [
        state_column, city_id_column, city_name_column,
        school_type_column, first_to_fifth_grade_results_column, first_grade_results_column, second_grade_results_column,
        third_grade_results_column, fourth_grade_results_column, fifth_grade_results_column, yield_indicator_column,
        math_results_column, portuguese_results_column, standardized_average_column, ideb_column
    ],
    [
        school_type_column, first_to_fifth_grade_results_column, first_grade_results_column, second_grade_results_column,
        third_grade_results_column, fourth_grade_results_column, fifth_grade_results_column, yield_indicator_column,
        math_results_column, portuguese_results_column, standardized_average_column, ideb_column
    ],
    [
        state_column, city_id_column, city_name_column,
        school_id_column, school_name_column, school_type_column,
        first_to_third_high_grade_results_column, first_high_grade_results_column,
        second_high_grade_results_column, third_high_grade_results_column, yield_indicator_column,
        math_results_column, portuguese_results_column, standardized_average_column, ideb_column
    ],
    [
        state_column, city_id_column, city_name_column, school_type_column,
        first_to_third_high_grade_results_column, first_high_grade_results_column,
        second_high_grade_results_column, third_high_grade_results_column, yield_indicator_column,
        math_results_column, portuguese_results_column, standardized_average_column, ideb_column
    ],
    [
        state_or_region, school_type_column,
        first_to_fifth_grade_results_column, first_grade_results_column, second_grade_results_column,
        third_grade_results_column, fourth_grade_results_column, fifth_grade_results_column, yield_indicator_column,
        math_results_column, portuguese_results_column, standardized_average_column, ideb_column
    ]
]

primary_keys = [

    [adm_dependence_column, education_column],
    [city_id_column, adm_dependence_column, education_column],
    [state_column, adm_dependence_column, education_column],
    [school_id_column],
    [city_id_column, school_type_column],
    [school_id_column],
    [city_id_column, school_type_column],
    [school_type_column],
    [school_id_column],
    [city_id_column, school_type_column],
    [state_or_region, school_type_column]

]

dtype = [

    {
        adm_dependence_column: 'string',
        education_column: 'string',
        teacher_number_column: 'int32',
        rais_teacher_percentage_column: 'float32',
        first_quartile_column: 'float32',
        median_column: 'float32',
        average_column: 'float32',
        third_quartile_column: 'float32',
        standard_deviation_column: 'float32' ,
        weekly_workload_column: 'float32',
        standardized_pay_40_hours_column: 'float32',
    },

    {

        region_column: 'string',
        state_column: 'string',
        city_id_column: 'int32',
        city_name_column: 'string',
        adm_dependence_column: 'string',
        education_column: 'string',
        teacher_number_column: 'int32',
        rais_teacher_percentage_column: 'float32',
        first_quartile_column: 'float32',
        median_column: 'float32',
        average_column: 'float32',
        third_quartile_column: 'float32',
        standard_deviation_column: 'float32' ,
        weekly_workload_column: 'float32',
        standardized_pay_40_hours_column: 'float32',
    },

    {
        region_column: 'string',
        state_column: 'string',
        adm_dependence_column: 'string',
        education_column: 'string',
        teacher_number_column: 'int32',
        rais_teacher_percentage_column: 'float32',
        first_quartile_column: 'float32',
        median_column: 'float32',
        average_column: 'float32',
        third_quartile_column: 'float32',
        standard_deviation_column: 'float32' ,
        weekly_workload_column: 'float32',
        standardized_pay_40_hours_column: 'float32',

    },

    {

        state_column: 'string',
        city_id_column: 'int32',
        city_name_column: 'string',
        school_id_column: 'int32',
        school_name_column: 'string',
        school_type_column: 'string',
        sixth_to_nineth_grade_results_column: 'float32',
        sixth_grade_results_column: 'float32',
        seventh_grade_results_column: 'float32',
        eighth_grade_results_column: 'float32',
        ninth_grade_results_column: 'float32',
        yield_indicator_column: 'float32',
        math_results_column: 'float32',
        portuguese_results_column: 'float32',
        standardized_average_column: 'float32',
        ideb_column: 'float32'

    },

    {

        state_column: 'string',
        city_id_column: 'int32',
        city_name_column: 'string',
        school_type_column: 'string',
        sixth_to_nineth_grade_results_column: 'float32',
        sixth_grade_results_column: 'float32',
        seventh_grade_results_column: 'float32',
        eighth_grade_results_column: 'float32',
        ninth_grade_results_column: 'float32',
        yield_indicator_column: 'float32',
        math_results_column: 'float32',
        portuguese_results_column: 'float32',
        standardized_average_column: 'float32',
        ideb_column: 'float32'

    },

    {

        state_column: 'string',
        city_id_column: 'int32',
        city_name_column: 'string',
        school_id_column: 'int32',
        school_name_column: 'string',
        school_type_column: 'string',
        first_to_fifth_grade_results_column: 'float32',
        first_grade_results_column: 'float32',
        second_grade_results_column: 'float32',
        third_grade_results_column: 'float32',
        fourth_grade_results_column: 'float32',
        fifth_grade_results_column: 'float32',
        yield_indicator_column: 'float32',
        math_results_column: 'float32',
        portuguese_results_column: 'float32',
        standardized_average_column: 'float32',
        ideb_column: 'float32'

    },

    {

        state_column: 'string',
        city_id_column: 'int32',
        city_name_column: 'string',
        school_type_column: 'string',
        first_to_fifth_grade_results_column: 'float32',
        first_grade_results_column: 'float32',
        second_grade_results_column: 'float32',
        third_grade_results_column: 'float32',
        fourth_grade_results_column: 'float32',
        fifth_grade_results_column: 'float32',
        yield_indicator_column: 'float32',
        math_results_column: 'float32',
        portuguese_results_column: 'float32',
        standardized_average_column: 'float32',
        ideb_column: 'float32'

    },

    {
        school_type_column: 'string',
        first_to_fifth_grade_results_column: 'float32',
        first_grade_results_column: 'float32',
        second_grade_results_column: 'float32',
        third_grade_results_column: 'float32',
        fourth_grade_results_column: 'float32',
        fifth_grade_results_column: 'float32',
        yield_indicator_column: 'float32',
        math_results_column: 'float32',
        portuguese_results_column: 'float32',
        standardized_average_column: 'float32',
        ideb_column: 'float32'

    },

    {

        state_column: 'string',
        city_id_column: 'int32',
        city_name_column: 'string',
        school_id_column: 'int32',
        school_name_column: 'string',
        school_type_column: 'string',
        first_to_third_high_grade_results_column: 'float32',
        first_high_grade_results_column: 'float32',
        second_high_grade_results_column: 'float32',
        third_high_grade_results_column: 'float32',
        yield_indicator_column: 'float32',
        math_results_column: 'float32',
        portuguese_results_column: 'float32',
        standardized_average_column: 'float32',
        ideb_column: 'float32'

    },

    {

        state_column: 'string',
        city_id_column: 'int32',
        city_name_column: 'string',
        school_type_column: 'string',
        first_to_third_high_grade_results_column: 'float32',
        first_high_grade_results_column: 'float32',
        second_high_grade_results_column: 'float32',
        third_high_grade_results_column: 'float32',
        yield_indicator_column: 'float32',
        math_results_column: 'float32',
        portuguese_results_column: 'float32',
        standardized_average_column: 'float32',
        ideb_column: 'float32'

    },

    {

        state_or_region: 'string',
        school_type_column: 'string',
        first_to_third_high_grade_results_column: 'float32',
        first_high_grade_results_column: 'float32',
        second_high_grade_results_column: 'float32',
        third_high_grade_results_column: 'float32',
        yield_indicator_column: 'float32',
        math_results_column: 'float32',
        portuguese_results_column: 'float32',
        standardized_average_column: 'float32',
        ideb_column: 'float32'

    },


]

table_names = [
    'wage_brasil',
    'wage_per_city',
    'wage_per_state',
    'primary_schools_final_years',
    'cities_primary_schools_final_years',
    'primary_schools_early_years',
    'cities_primary_schools_early_years',
    'brasil_primary_schools_early_years',
    'high_schools',
    'cities_high_schools',
    'states_region_primary_schools_early_years'
]


intervals = [
    [('A', 'B')], [('A', 'A')], [('A', 'A')],[('G', 'AV'), ('BC', 'BW'), ('CA', 'CG'), ('CI', 'CP')],
    [('E', 'AT'), ('BA', 'BU'), ('BY', 'CE'), ('CG', 'CN')],
    [('G', 'BC'), ('BK', 'CE'), ('CI', 'CO'), ('CQ', 'CX')],
    [('E', 'BA'), ('BI', 'CC'), ('CG', 'CM'), ('CO', 'CV')],
    [('A', 'A'), ('C', 'AY'), ('BG', 'CA'), ('CE', 'CK'), ('CM', 'CT')],
    [('G', 'L'), ('Q', 'Q'), ('S', 'U'), ('Y', 'Y'), ('AA', 'AB')],
    [('E', 'J'), ('O', 'O'), ('Q', 'S'), ('W', 'W'), ('Y', 'Z')],
    [('C', 'AY'), ('BG', 'CA'), ('CE', 'CK'), ('CM', 'CT')]
]

# Função para converter letra para número de coluna no Excel
def char_to_number(letter):
    number = 0
    for i, l in enumerate(reversed(letter)):
        number += (ord(l) - 64) * (26 ** i)
    return number - 1

# Array para armazenar os índices das colunas
drop_columns = []

# Percorrendo os intervalos e adicionando os índices das colunas ao array

for interval in intervals:
    temp = []
    for begin, end in interval:
        index_begin = char_to_number(begin)
        index_end = char_to_number(end)
        temp.extend(range(index_begin, index_end + 1))
    drop_columns.append(temp)

skip_rows = [[9, 3], [9, 3], [9, 3], [10, 3], [10, 3], [10, 3], [10, 3], [10, 3], [10, 3], [10, 3], [10, 6]]

unwanted = ['a', 'c', 'd', '-']

for i in range(0, 11) :
    dataframe = pd.read_excel(f'{extract_dir}{extracted_files[i]}', skiprows=skip_rows[i][0], header=None, skipfooter=skip_rows[i][1], dtype=dtype[i])

    columns_to_drop = dataframe.columns[drop_columns[i]]
    dataframe = dataframe.drop(columns_to_drop, axis=1)

    dataframe.columns = columns[i]
    

    dataframe_filtered = dataframe.dropna()

    mask_unwanted = dataframe.isin(unwanted).any(axis=1)
    dataframe_filtered = dataframe_filtered[~mask_unwanted]

    if (i == 7) or (i == 10):
        dataframe_filtered.loc[dataframe_filtered[school_type_column].str.contains(r'Privada \(\d\)', regex=True), school_type_column] = 'Privada'
        dataframe_filtered.loc[dataframe_filtered[school_type_column].str.contains(r'Estadual \(\d\)', regex=True), school_type_column] = 'Estadual'
        dataframe_filtered.loc[dataframe_filtered[school_type_column].str.contains(r'Pública \(\d\)', regex=True), school_type_column] = 'Pública'
        dataframe_filtered.loc[dataframe_filtered[school_type_column].str.contains(r'Municipal \(\d\)', regex=True), school_type_column] = 'Municipal'
        dataframe_filtered.loc[dataframe_filtered[school_type_column].str.contains(r'Total \(\d\)\(\d\)', regex=True), school_type_column] = 'Total'
        dataframe_filtered.loc[dataframe_filtered[school_type_column].str.contains(r'Total \(\d\)', regex=True), school_type_column] = 'Total'

    if (i == 0) or (i == 2):
        dataframe_filtered[standardized_pay_40_hours_column] = dataframe_filtered[standardized_pay_40_hours_column].astype(str).str.replace(',', '', regex=False)

    dataframe_filtered.to_sql(table_names[i], conn, index=False, if_exists='replace')

    dataframe_filtered.set_index(primary_keys[i], inplace=True)
    
    print(f'{table_names[i]} table created')

print('Import completed')
conn.close()

# Data Analysis

In [ ]:


def display_df(df, query):
    table = PrettyTable()
    table.field_names = df.columns.tolist()
    for row in df.itertuples(index=False, name=None):
        table.add_row(row)
    print(f"Results for query: {query}\n")
    print(table)
    print("\n")

# Path to your .db file
database_dir = './data/database.db'

# Establish a connection to the SQLite database
conn = sqlite3.connect(database_dir)

# Example query to fetch data from a table
query = "SELECT * FROM wage_brasil"

# Execute the query and load the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

display_df(df, query)

# Close the connection
conn.close()
